SECTION 1: LOAD DATA CSVs


In [1]:
import os
import pandas as pd

In [2]:
#LOAD DFs for each campaign
ORGANIZATION = "DOE"
CAMPAIGN_NAME_LIST = ['ACE-ENA', 'ACMEV', 'BBOP', 'CACTI', 'CARES', 'GOAMAZON', 'ISDAC', 'TCAP2012', 'TCAP2013']

COLUMN_NAME_LIST_PATH = rf"C:\Users\haika\Desktop\May_Research\may_datasets\column_name_list"
RAW_PATH = rf"C:\Users\haika\Desktop\May_Research\may_datasets\raw_campaigns"

df_dict = {}

# Populate df_dict with campaign name as key and dataframe as value
for campaign_name in CAMPAIGN_NAME_LIST:
   file_path = os.path.join(RAW_PATH, f"{campaign_name}_raw.csv")
   df_dict[campaign_name] = pd.read_csv(file_path)
   print(f"Loaded {campaign_name}: {df_dict[campaign_name].shape}")

Loaded ACE-ENA: (546787, 101)
Loaded ACMEV: (549013, 384)
Loaded BBOP: (454849, 436)
Loaded CACTI: (385701, 730)


C:\Users\haika\AppData\Local\Temp\ipykernel_16468\873514819.py:13: DtypeWarning: Columns (30,61,165,202) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[campaign_name] = pd.read_csv(file_path)


Loaded CARES: (213996, 210)
Loaded GOAMAZON: (331893, 215)


C:\Users\haika\AppData\Local\Temp\ipykernel_16468\873514819.py:13: DtypeWarning: Columns (674,676) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[campaign_name] = pd.read_csv(file_path)


Loaded ISDAC: (441569, 720)


C:\Users\haika\AppData\Local\Temp\ipykernel_16468\873514819.py:13: DtypeWarning: Columns (5,40,41,107,111) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[campaign_name] = pd.read_csv(file_path)


Loaded TCAP2012: (150080, 121)


C:\Users\haika\AppData\Local\Temp\ipykernel_16468\873514819.py:13: DtypeWarning: Columns (2,9,78,80,184,188) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[campaign_name] = pd.read_csv(file_path)


Loaded TCAP2013: (187783, 224)


In [3]:
#LOAD COLUMN NAME LIST for each campaign
column_name_dict = {}

# Populate column_name_dict with campaign name as key and name mapping dict as value
for campaign_name in CAMPAIGN_NAME_LIST:
   file_path = os.path.join(COLUMN_NAME_LIST_PATH, f"{campaign_name}.csv")
   
   # Read the CSV file
   name_mapping_df = pd.read_csv(file_path)
   
   # Create dictionary mapping from 'original' to 'new' columns
   name_map = dict(zip(name_mapping_df['original'], name_mapping_df['new']))
   
   # Store in column_name_dict
   column_name_dict[campaign_name] = name_map
   
   print(f"Loaded {campaign_name} name mapping: {len(name_map)} columns")


Loaded ACE-ENA name mapping: 66 columns
Loaded ACMEV name mapping: 139 columns
Loaded BBOP name mapping: 143 columns
Loaded CACTI name mapping: 142 columns
Loaded CARES name mapping: 135 columns
Loaded GOAMAZON name mapping: 152 columns
Loaded ISDAC name mapping: 144 columns
Loaded TCAP2012 name mapping: 52 columns
Loaded TCAP2013 name mapping: 151 columns


In [4]:
#RENAME FILES AND SAVE TO NEW PATH

# Define output path for renamed files
RENAMED_OUTPUT_PATH = rf'C:\Users\haika\Desktop\May_Research\may_datasets\renamed_data'

# Process each campaign
for campaign_name in CAMPAIGN_NAME_LIST:
   print(f"\nProcessing {campaign_name}...")
   
   # Get the dataframe and column mapping for this campaign
   df_campaign = df_dict[campaign_name].copy()  # Make a copy to avoid modifying original
   
   # Handle case-insensitive duplicate columns
   print("Checking for case-insensitive duplicate columns...")
   column_groups = {}
   for col in df_campaign.columns:
       lower_col = col.lower()
       if lower_col not in column_groups:
           column_groups[lower_col] = []
       column_groups[lower_col].append(col)
   
   columns_to_drop = []
   for lower_col, col_list in column_groups.items():
       if len(col_list) > 1:
           print(f"Combining case-insensitive duplicates: {col_list}")
           primary_col = col_list[0]
           combined_series = df_campaign[col_list[0]].copy()
           for col in col_list[1:]:
               combined_series = combined_series.combine_first(df_campaign[col])
           df_campaign[primary_col] = combined_series
           columns_to_drop.extend(col_list[1:])
   
   if columns_to_drop:
       print(f"Dropping duplicate columns: {columns_to_drop}")
       df_campaign = df_campaign.drop(columns=columns_to_drop)
   
   raw_name_map = column_name_dict[campaign_name]
   
   # Separate mappings based on whether 'original' is empty or not
   valid_name_map = {}  # For actual column renaming
   all_new_columns = []  # All new column names we want in final df
   
   for old_col, new_col in raw_name_map.items():
       # Add new column to our desired list (if it's not empty)
       if pd.notna(new_col) and str(new_col).strip() != '':
           all_new_columns.append(new_col)
           
           # Only add to renaming map if old_col exists and is not empty
           if pd.notna(old_col) and str(old_col).strip() != '':
               # Check if the old column actually exists in the dataframe
               if old_col in df_campaign.columns:
                   valid_name_map[old_col] = new_col
               else:
                   print(f"MESSAGE: {campaign_name} is missing old column: {old_col}")
   
   print(f"Original columns: {len(df_campaign.columns)}")
   print(f"Valid mappings for renaming: {len(valid_name_map)}")
   print(f"Total new columns desired: {len(all_new_columns)}")
   
   # Rename columns using the valid mappings only
   df_renamed = df_campaign.rename(columns=valid_name_map)
   
   # Get the number of rows from the renamed dataframe
   num_rows = len(df_renamed)

   # Create final dataframe with ALL desired new columns
   df_final = pd.DataFrame()

   # Add Organization and Campaign columns first - with correct length
   df_final['Organization'] = [ORGANIZATION] * num_rows
   df_final['Campaign'] = [campaign_name] * num_rows
   
   # Then add all the mapped/renamed columns
   for new_col in all_new_columns:
       if new_col in df_renamed.columns:
           # Column exists after renaming, use it
           df_final[new_col] = df_renamed[new_col]
       else:
           # Column doesn't exist, create empty column
           df_final[new_col] = pd.NA
   
   print(f"Final columns (including Organization, Campaign, and empty ones): {len(df_final.columns)}")
   print(f"Empty columns created: {len([col for col in all_new_columns if col not in df_renamed.columns])}")
   
   # Create output filename
   output_filename = f"{campaign_name}_renamed.csv"
   output_path = os.path.join(RENAMED_OUTPUT_PATH, output_filename)
   
   # Save to CSV
   df_final.to_csv(output_path, index=False)
   print(f"Saved: {output_filename}")

print("\nAll campaigns processed!")


Processing ACE-ENA...
Checking for case-insensitive duplicate columns...
Original columns: 101
Valid mappings for renaming: 65
Total new columns desired: 66
Final columns (including Organization, Campaign, and empty ones): 68
Empty columns created: 1
Saved: ACE-ENA_renamed.csv

Processing ACMEV...
Checking for case-insensitive duplicate columns...
Original columns: 384
Valid mappings for renaming: 138
Total new columns desired: 139


C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[new_col] = df_renamed[new_col]
C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[new_col] = df_renamed[new_col]
C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

Final columns (including Organization, Campaign, and empty ones): 141
Empty columns created: 1
Saved: ACMEV_renamed.csv

Processing BBOP...
Checking for case-insensitive duplicate columns...
MESSAGE: BBOP is missing old column: ccn_SS_A
MESSAGE: BBOP is missing old column: ccn_CCN_Conc_A
MESSAGE: BBOP is missing old column: cpc_CPC_Conc_3010
Original columns: 436
Valid mappings for renaming: 139
Total new columns desired: 143


C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[new_col] = df_renamed[new_col]
C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[new_col] = df_renamed[new_col]
C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

Final columns (including Organization, Campaign, and empty ones): 145
Empty columns created: 4
Saved: BBOP_renamed.csv

Processing CACTI...
Checking for case-insensitive duplicate columns...
Original columns: 730
Valid mappings for renaming: 141
Total new columns desired: 142


...

Final columns (including Organization, Campaign, and empty ones): 154
Empty columns created: 1
Saved: GOAMAZON_renamed.csv

Processing ISDAC...
Checking for case-insensitive duplicate columns...
Combining case-insensitive duplicates: ['splat-concentration_Concentration_p/cc', 'splat-concentration_concentration_p/cc']
Dropping duplicate columns: ['splat-concentration_concentration_p/cc']
Original columns: 719
Valid mappings for renaming: 143
Total new columns desired: 144


C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[new_col] = df_renamed[new_col]
C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[new_col] = df_renamed[new_col]
C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

Final columns (including Organization, Campaign, and empty ones): 146
Empty columns created: 1
Saved: ISDAC_renamed.csv

Processing TCAP2012...
Checking for case-insensitive duplicate columns...
Combining case-insensitive duplicates: ['sp2_ALT_m', 'sp2_Alt_m']
Dropping duplicate columns: ['sp2_Alt_m']
Original columns: 120
Valid mappings for renaming: 51
Total new columns desired: 52
Final columns (including Organization, Campaign, and empty ones): 54
Empty columns created: 1
Saved: TCAP2012_renamed.csv

Processing TCAP2013...
Checking for case-insensitive duplicate columns...
Combining case-insensitive duplicates: ['sp2_Mass_ng_m3', 'sp2_mass_ng_m3']
Dropping duplicate columns: ['sp2_mass_ng_m3']
Original columns: 223
Valid mappings for renaming: 150
Total new columns desired: 151


C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[new_col] = df_renamed[new_col]
C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[new_col] = df_renamed[new_col]
C:\Users\haika\AppData\Local\Temp\ipykernel_16468\615157742.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

Final columns (including Organization, Campaign, and empty ones): 153
Empty columns created: 1
Saved: TCAP2013_renamed.csv

All campaigns processed!
